In [ ]:
# SOURCE
SOURCE = "https://www.tesladeaths.com"


In [ ]:
from requests_html import HTMLSession, Element

session = HTMLSession()
r: session = session.get(SOURCE)


Getting the `table` element that houses the data.



In [ ]:
table: Element = r.html.find("#dttable", first=True)
table


Finding all row headers using `th` tag.



In [ ]:
table_headers = table.find("th")
table_headers = [row.text for row in table_headers]
table_headers


Collecting all data rows and excluding the last 350 since there is no usable data in there.



In [ ]:
table_data = table.find("td")[:-350]
# * The last 350 td"s do not have useful data

for data in table_data:
    print(data.text)


Since I used `text` element to find the content, the `URL`s got truncated. From the table component, I found `a` html tags and used their `href` attributes to extract the links.



In [ ]:
table_links = table.find("a")
urls = [link.attrs["href"] for link in table_links]
# urls


Looped through the data I found and created a list of **23** values, since we have 23 table headers. Essentially, within the `row` variable is a list of lists, where each individual list is a row of the "table".



In [ ]:
rows = []
for i in list(range(0, len(table_data), 23)):
    one_row = [data.text for data in table_data[0 + i : 23 + i]]
    rows.append(one_row)

for url in urls:
    for row in rows:
        row[17] = url

# rows


Mandatory conversion to DataFrame 😅


In [ ]:
import pandas as pd

df = pd.DataFrame(rows, columns=table_headers)
df.to_csv("./data.csv")
df.head()


Played around with the new library `Polars` 🐻‍❄️ which is supposed to the `fast as fuck boiiiii`. It is indeed, or should be in theory. `Polars` store data in DataFrames in _columnar_ format as opposed to the classical row format _Pandas_ 🐼 uses.


In [189]:
# import polars as pl
# pl_rows = pl.DataFrame(rows)
# pl_rows


Converted data for certain values from `str` to `int` so that some analysis can be performed 📊


In [192]:
# Converting necessary columns from str to int values
int_value_columns = df.columns[6:15]
df[int_value_columns] = (
    df[int_value_columns].apply(pd.to_numeric, errors="coerce").fillna(0).astype("int")
)

# Confirming one column
df["Deaths"].dtype


dtype('int64')

In [193]:
df.head()


,Case #,Year,Date,Country,State,Description,Deaths,Tesla driver,Tesla occupant,Other vehicle,...,AutoPilot claimed,Reported in NHTSA SGO,Verified Tesla Autopilot Death,Excerpt Verifying Tesla Autopilot Deaths,Source,Note,Deceased 1,Deceased 2,Deceased 3,Deceased 4
0,377,2023,11/15/2023,USA,NC,Moped runs into Tesla,1,0,0,1,...,0,0,-,-,https://web.archive.org/web/20150425055520/htt...,,,,,
1,376,2023,11/4/2023,USA,WA,Tesla rolls over and hits enbankment,1,1,0,0,...,0,0,-,-,https://web.archive.org/web/20150425055520/htt...,,,,,
2,375,2023,11/3/2023,USA,FL,Tesla hits a dislodged motorcyclist,1,0,0,0,...,0,0,-,-,https://web.archive.org/web/20150425055520/htt...,,,,,
3,374,2023,10/30/2023,USA,NY,Tesla hits multiple cars,1,0,1,0,...,0,0,-,-,https://web.archive.org/web/20150425055520/htt...,,,,,
4,373,2023,10/30/2023,USA,NY,DUI,1,0,0,1,...,0,0,-,-,https://web.archive.org/web/20150425055520/htt...,,,,,
